In [1]:
import re
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
BASE_URL = "https://meeyland.com/mua-ban-nha-dat-ho-chi-minh-b43?page={}"
MAX_PAGE = 2
MAX_ITEMS = 1250
SAVE_EVERY = 250
CSV_NAME = "meeyland_hcm.csv"

FIELDS = [
    "id","Page","Title","Price_Raw","Price_Billion","Area_m2",
    "District","Address","Bedrooms","Toilets",
    "Post_Time","Link","Description"
]

In [3]:
def init_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

In [4]:
def clean_district(text):
    if not text:
        return ""
    text = text.lower()
    if "thủ đức" in text:
        return "Thủ Đức"
    m = re.search(r"quận\s*(\d+)", text)
    if m:
        return f"Quận {m.group(1)}"
    for d in [
        "bình thạnh","tân bình","tân phú","phú nhuận","gò vấp",
        "bình tân","nhà bè","hóc môn","củ chi","cần giờ","bình chánh"
    ]:
        if d in text:
            return d.title()
    return ""

def price_to_billion(price):
    if not price:
        return ""
    price = price.replace(".", "").replace(",", "")
    m = re.search(r"(\d+)", price)
    return round(int(m.group(1)) / 1_000_000_000, 2) if m else ""

In [5]:
def scrape_detail(driver, url, page):
    driver.get(url)
    time.sleep(random.uniform(2, 3))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    canonical = soup.find("link", rel="canonical")
    link = canonical["href"] if canonical else url
    ad_id = link.rstrip("/").split("/")[-1]

    title = soup.find("meta", property="og:title")
    title = title["content"] if title else ""

    desc_meta = soup.find("meta", {"name": "description"})
    address = desc_meta["content"] if desc_meta else ""

    post_time = ""
    m = re.search(r"Ngày đăng:\s*(\d{2}/\d{2}/\d{4})", soup.text)
    if m:
        post_time = m.group(1)

    area = ""
    m = re.search(r"(\d+)\s?m²", soup.text)
    if m:
        area = m.group(1)

    price_raw = ""
    m = re.search(r"giá\s*([\d\.]+)\s*VNĐ", soup.text)
    if m:
        price_raw = m.group(1)

    description = ""
    body = soup.find("astro-island", {"component-export": "default"})
    if body:
        description = body.get_text(" ", strip=True)

    return {
        "id": ad_id,
        "Page": page,
        "Title": title,
        "Price_Raw": price_raw,
        "Price_Billion": price_to_billion(price_raw),
        "Area_m2": area,
        "District": clean_district(address),
        "Address": address,
        "Bedrooms": "",
        "Toilets": "",
        "Post_Time": post_time,
        "Link": link,
        "Description": description
    }

In [6]:
def main():
    driver = init_driver()
    results = []
    seen_ids = set()

    for page in range(1, MAX_PAGE + 1):
        if len(results) >= MAX_ITEMS:
            break

        driver.get(BASE_URL.format(page))
        time.sleep(random.uniform(2, 3))

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = [
            "https://meeyland.com" + a["href"]
            for a in soup.select("a[href*='/ban-']")
        ]

        for link in links:
            if len(results) >= MAX_ITEMS:
                break

            try:
                item = scrape_detail(driver, link, page)
                if item["id"] in seen_ids:
                    continue
                seen_ids.add(item["id"])
                results.append(item)
                print(f"✔ {item['id']} | {len(results)}")

                if len(results) % SAVE_EVERY == 0:
                    pd.DataFrame(results, columns=FIELDS)\
                      .to_csv(CSV_NAME, index=False, encoding="utf-8-sig")
                    print("💾 SAVED CSV")

            except Exception as e:
                print("❌ FAIL:", link, e)

        time.sleep(random.uniform(3, 5))

    pd.DataFrame(results, columns=FIELDS)\
      .to_csv(CSV_NAME, index=False, encoding="utf-8-sig")

    driver.quit()
    print("✅ DONE:", len(results))

In [7]:
if __name__ == "__main__":
    main()

✔ ban-can-ho-chung-cu-g14 | 1
✔ ban-chung-cu-mini-g13 | 2
✔ ban-nha-rieng-g12 | 3
✔ ban-nha-mat-pho-g13 | 4
✔ ban-nha-tap-the-g13 | 5
✔ ban-dat-g11 | 6
✔ ban-dat-nen-du-an-g14 | 7
✔ ban-biet-thu-lien-ke-g14 | 8
✔ ban-shophouse-nha-pho-thuong-mai-g15 | 9
✔ ban-nha-tro-g12 | 10
✔ 306099833 | 11
✔ 305966371 | 12
✔ 303270749 | 13
✔ 306216741 | 14
✔ 305779557 | 15
✔ 306216521 | 16
✔ 306068058 | 17
✔ 306048852 | 18
✔ 306057091 | 19
✔ 306216130 | 20
✔ 306057087 | 21
✔ 306058345 | 22
✔ 306159248 | 23
✔ 305694767 | 24
✔ 305679507 | 25
✔ 305875815 | 26
✔ 306216560 | 27
✔ 306057627 | 28
✔ 306068169 | 29
✔ 306068096 | 30
✔ 306068101 | 31
✔ 305915817 | 32
✔ 305915781 | 33
✔ 305915866 | 34
✔ 306068566 | 35
✔ 306049675 | 36
✔ 306049678 | 37
✔ 306068285 | 38
✔ 306068296 | 39
✔ 306216360 | 40
✔ ban-nha-rieng-tuyen-quang-h122 | 41
✔ ban-nha-rieng-lao-cai-h122 | 42
✔ ban-nha-rieng-thai-nguyen-h122 | 43
✔ ban-nha-rieng-phu-tho-h122 | 44
✔ ban-nha-rieng-bac-ninh-h122 | 45
✔ ban-nha-rieng-hung-yen-h122 | 46

In [15]:
import html
import json
import re
import time
import random
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ================= CONFIG =================
BASE_LIST_URL = "https://meeyland.com/mua-ban-nha-dat-ho-chi-minh-b43?page={}"
START_PAGE = 1
END_PAGE = 50
SAVE_EVERY = 250
OUTPUT_PREFIX = "meeyland_hcm_part"

# ================= DRIVER =================
def init_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

# ================= UTILS =================
def extract_id(link):
    m = re.search(r"/(\d{6,})$", link)
    return m.group(1) if m else None

def price_to_billion(text):
    if not text:
        return None
    t = text.lower().replace(",", ".")
    m = re.search(r"([\d\.]+)", t)
    return float(m.group(1)) if m else None

def clean_area(text):
    if not text:
        return None
    t = text.lower()
    if "/" in t or "triệu" in t:
        return None
    m = re.search(r"([\d\.]+)\s*m", t)
    return float(m.group(1)) if m else None

# ================= DISTRICT FROM ADDRESS =================
def extract_district(address):
    if not address:
        return None

    addr = address.lower()

    # ===== TP Thủ Đức =====
    if re.search(r"(tp\.?\s*thủ\s*đức)", addr) or re.search(r"(thành\s*phố\s*thủ\s*đức)", addr):
        return "TP Thủ Đức"

    # ===== Quận =====
    m = re.search(r"(q\.?\s*\d+)", addr)
    if m:
        return m.group(1).replace("q.", "Quận ").replace("q", "Quận ").strip().title()

    m = re.search(r"(quận\s*\d+)", addr)
    if m:
        return m.group(1).title()

    m = re.search(r"(quận\s*[a-zà-ỹ\s]+)", addr)
    if m:
        return m.group(1).title()

    # ===== Huyện =====
    m = re.search(r"(h\.?\s*[a-zà-ỹ\s]+)", addr)
    if m:
        name = m.group(1)
        name = name.replace("h.", "").replace("h ", "").strip()
        return f"Huyện {name.title()}"

    m = re.search(r"(huyện\s*[a-zà-ỹ\s]+)", addr)
    if m:
        return m.group(1).title()

    return None

# ================= DETAIL =================
def crawl_detail(driver, url):
    driver.get(url)
    time.sleep(random.uniform(2.2, 3))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ========= CANONICAL =========
    canonical = soup.find("link", rel="canonical")
    link = canonical["href"] if canonical else url

    # ========= POST TIME =========
    post_time = None
    p = soup.find("p", string=re.compile("Ngày đăng"))
    if p:
        post_time = p.get_text(strip=True).replace("Ngày đăng:", "").strip()

    # ========= TITLE =========
    title = soup.find("h1").get_text(strip=True) if soup.find("h1") else None

    # ========= PRICE =========
    price_raw = None
    h2 = soup.select_one("h2.text-error-600")
    if h2:
        price_raw = h2.get_text(strip=True)

    price_billion = price_to_billion(price_raw)

    # ========= AREA / PN / WC =========
    area_m2 = bedrooms = toilets = None
    spans = soup.select("span.text-fs-14")

    for sp in spans:
        t = sp.get_text(strip=True).lower()

        if area_m2 is None:
            area = clean_area(t)
            if area:
                area_m2 = area

        if "pn" in t:
            m = re.search(r"\d+", t)
            if m:
                bedrooms = int(m.group())

        if "wc" in t:
            m = re.search(r"\d+", t)
            if m:
                toilets = int(m.group())

    # ========= ADDRESS =========
    address = None
    addr_div = soup.select_one("div.text-fs-14.font-medium.text-primary-600")
    if addr_div:
        address = addr_div.get_text(strip=True)

    # ========= DISTRICT =========
    district = extract_district(address)

    # ========= DESCRIPTION =========
    description = None
    for astro in soup.find_all("astro-island"):
        props = astro.get("props")
        if not props:
            continue
        try:
            data = json.loads(html.unescape(props))
            if "article" in data:
                desc_html = data["article"].get("description")
                if desc_html:
                    description = BeautifulSoup(
                        desc_html, "html.parser"
                    ).get_text(" ", strip=True)
        except:
            continue

    return {
        "Title": title,
        "Price_Raw": price_raw,
        "Price_Billion": price_billion,
        "Area_m2": area_m2,
        "District": district,
        "Address": address,
        "Bedrooms": bedrooms,
        "Toilets": toilets,
        "Post_Time": post_time,
        "Link": link,
        "Description": description
    }

# ================= MAIN =================
def crawl():
    driver = init_driver()
    all_rows = []
    seen_ids = set()
    part = 1

    try:
        for page in range(START_PAGE, END_PAGE + 1):
            print(f"\n➡️ Page {page}")
            driver.get(BASE_LIST_URL.format(page))
            time.sleep(random.uniform(2, 3))

            soup = BeautifulSoup(driver.page_source, "html.parser")

            links = set()
            for a in soup.select("a[href]"):
                href = a["href"]
                if "ho-chi-minh" in href and re.search(r"/\d{6,}$", href):
                    links.add("https://meeyland.com" + href)

            print(f"   Found {len(links)} links")

            for link in links:
                ad_id = extract_id(link)
                if not ad_id or ad_id in seen_ids:
                    continue

                print(f"✔ {ad_id}")
                seen_ids.add(ad_id)

                detail = crawl_detail(driver, link)
                all_rows.append({
                    "id": ad_id,
                    "Page": page,
                    **detail
                })

                if len(all_rows) % SAVE_EVERY == 0:
                    pd.DataFrame(all_rows).to_csv(
                        f"{OUTPUT_PREFIX}_{part}.csv",
                        index=False,
                        encoding="utf-8-sig"
                    )
                    print(f"💾 Saved {len(all_rows)} rows")
                    part += 1

    finally:
        driver.quit()

    pd.DataFrame(all_rows).to_csv(
        f"{OUTPUT_PREFIX}_final.csv",
        index=False,
        encoding="utf-8-sig"
    )
    print(f"\n✅ DONE — {len(all_rows)} rows")

# ================= RUN =================
if __name__ == "__main__":
    crawl()



➡️ Page 1
   Found 30 links
✔ 303270749
✔ 306068169
✔ 306059417
✔ 306057087
✔ 306216130
✔ 305875815
✔ 306068101
✔ 306099833
✔ 306049675
✔ 306068566
✔ 306216741
✔ 305915866
✔ 305966371
✔ 306057627
✔ 305915817
✔ 306048852
✔ 305915781
✔ 306057091
✔ 306058345
✔ 306216560
✔ 306068058
✔ 306059752
✔ 306074379
✔ 306068096
✔ 306159248
✔ 305679507
✔ 305779557
✔ 305694767
✔ 306216521
✔ 306060772

➡️ Page 2
   Found 30 links
✔ 306074042
✔ 302331677
✔ 306074083
✔ 306057624
✔ 306215796
✔ 306069211
✔ 306215781
✔ 306050119
✔ 306068914
✔ 306215807
✔ 306049801
✔ 302376995
✔ 306068894
✔ 306068296
✔ 306069207
✔ 305931411
✔ 302294827
✔ 306068631
✔ 306055056
✔ 302405902
✔ 306049678
✔ 306069232
✔ 306216039
✔ 306068285
✔ 306049697
✔ 306215456
✔ 302358123
✔ 306216360
✔ 306055349
✔ 306055535

➡️ Page 3
   Found 30 links
✔ 305225557
✔ 306089231
✔ 302387485
✔ 306211250
✔ 306215831
✔ 303527309
✔ 306051672
✔ 303033117
✔ 306051254
✔ 306069505
✔ 303877793
✔ 305301643
✔ 303331336
✔ 306211896
✔ 306212054
✔ 303260358
✔

In [16]:
import html
import json
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed

# ================= CONFIG =================
INPUT_CSV = "meeyland_hcm_part_final.csv"
OUTPUT_CSV = "meeyland_hcm_fixed_thread.csv"
MAX_THREADS = 4  # số driver chạy song song
SLEEP_MIN = 2.2
SLEEP_MAX = 3.2

# ================= DRIVER =================
def init_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

# ================= DETAIL FIX =================
def fetch_address_description(link):
    driver = init_driver()
    try:
        driver.get(link)
        time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX))
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ===== ADDRESS (DOM) =====
        addr_div = soup.select_one("div.text-fs-14.font-medium.text-primary-600")
        address = addr_div.get_text(strip=True) if addr_div else None

        # ===== DESCRIPTION (JSON) =====
        description = None
        for astro in soup.find_all("astro-island"):
            props = astro.get("props")
            if not props:
                continue
            try:
                data = json.loads(html.unescape(props))
                article = data.get("article")
                if article and article.get("description"):
                    description = BeautifulSoup(
                        article["description"], "html.parser"
                    ).get_text(" ", strip=True)
                    break
            except:
                continue
        return link, address, description
    finally:
        driver.quit()

# ================= MAIN =================
def main():
    df = pd.read_csv(INPUT_CSV)
    rows_to_fix = df[pd.isna(df["Address"]) | pd.isna(df["Description"])]

    print(f"🔧 Total rows to fix: {len(rows_to_fix)}")

    results = []

    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        future_to_link = {executor.submit(fetch_address_description, link): idx for idx, link in rows_to_fix["Link"].items()}

        for future in as_completed(future_to_link):
            idx = future_to_link[future]
            try:
                link, address, description = future.result()
                if address:
                    df.at[idx, "Address"] = address
                if description:
                    df.at[idx, "Description"] = description
                print(f"✔ Fixed row {idx} | {link}")
            except Exception as e:
                print(f"⚠️ Failed row {idx}: {e}")

    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"\n✅ DONE — Saved to {OUTPUT_CSV}")

# ================= RUN =================
if __name__ == "__main__":
    main()

🔧 Total rows to fix: 1488
✔ Fixed row 2 | https://meeyland.com/ban-dat-tan-binh-ho-chi-minh-i1123/306059417
✔ Fixed row 1 | https://meeyland.com/ban-nha-rieng-tan-binh-ho-chi-minh-i1223/306068169
✔ Fixed row 0 | https://meeyland.com/ban-nha-rieng-nha-be-ho-chi-minh-i1223/303270749
✔ Fixed row 3 | https://meeyland.com/ban-nha-rieng-quan-12-ho-chi-minh-i1223/306057087
✔ Fixed row 5 | https://meeyland.com/ban-nha-rieng-quan-3-ho-chi-minh-i1223/305875815
✔ Fixed row 7 | https://meeyland.com/ban-nha-mat-pho-thu-duc-ho-chi-minh-i1323/306099833
✔ Fixed row 4 | https://meeyland.com/ban-can-ho-chung-cu-quan-7-ho-chi-minh-i1423/306216130
✔ Fixed row 6 | https://meeyland.com/ban-nha-rieng-binh-thanh-ho-chi-minh-i1223/306068101
✔ Fixed row 9 | https://meeyland.com/ban-nha-rieng-phu-nhuan-ho-chi-minh-i1223/306068566
✔ Fixed row 11 | https://meeyland.com/ban-nha-rieng-tan-phu-ho-chi-minh-i1223/305915866
✔ Fixed row 8 | https://meeyland.com/ban-nha-rieng-binh-tan-ho-chi-minh-i1223/306049675
✔ Fixed r

In [9]:
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [10]:
INPUT_CSV  = "meeyland_hcm_fixed_thread.csv"
OUTPUT_CSV = "meeyland_hcm_desc_final.csv"

URL_COL  = "Link"
DESC_COL = "Description"

MAX_WORKERS   = 4        # an toàn cho Meeyland
SAVE_EVERY    = 50       # lưu tạm sau mỗi 50 link
WAIT_TIMEOUT  = 10


In [11]:
def create_driver():
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--window-position=-10000,0")
    options.add_argument("--window-size=1200,800")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")

    return webdriver.Chrome(options=options)


In [12]:
def crawl_description(url):
    driver = create_driver()

    try:
        driver.get(url)

        WebDriverWait(driver, WAIT_TIMEOUT).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.article-description")
            )
        )

        # Click Xem thêm / Thu gọn
        try:
            btn = driver.find_element(
                By.XPATH,
                "//span[contains(text(),'Xem thêm') or contains(text(),'Thu gọn')]"
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.5)
        except:
            pass

        description = driver.find_element(
            By.CSS_SELECTOR,
            "div.article-description div.break-words"
        ).text.strip()

        return description

    except Exception:
        return None

    finally:
        driver.quit()

In [13]:
df = pd.read_csv(INPUT_CSV)

df[DESC_COL] = df[DESC_COL].fillna("").astype(str)

df_need = df[df[DESC_COL].str.strip() == ""]
urls = df_need[URL_COL].tolist()

print(f"Tổng link cần crawl: {len(urls)}")

Tổng link cần crawl: 1488


In [14]:
results = {}
count = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_url = {
        executor.submit(crawl_description, url): url
        for url in urls
    }

    for future in as_completed(future_to_url):
        url = future_to_url[future]
        desc = future.result()

        count += 1

        if desc:
            df.loc[df[URL_COL] == url, DESC_COL] = desc
            print(f"[OK]   {count}/{len(urls)}")
        else:
            print(f"[FAIL] {count}/{len(urls)}")

        if count % SAVE_EVERY == 0:
            df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
            print(f"💾 Đã lưu tạm {count} records")

[OK]   1/1488
[OK]   2/1488
[OK]   3/1488
[OK]   4/1488
[OK]   5/1488
[OK]   6/1488
[OK]   7/1488
[OK]   8/1488
[OK]   9/1488
[OK]   10/1488
[OK]   11/1488
[OK]   12/1488
[OK]   13/1488
[OK]   14/1488
[OK]   15/1488
[OK]   16/1488
[OK]   17/1488
[OK]   18/1488
[OK]   19/1488
[OK]   20/1488
[OK]   21/1488
[OK]   22/1488
[OK]   23/1488
[OK]   24/1488
[OK]   25/1488
[OK]   26/1488
[OK]   27/1488
[OK]   28/1488
[OK]   29/1488
[OK]   30/1488
[OK]   31/1488
[OK]   32/1488
[OK]   33/1488
[OK]   34/1488
[OK]   35/1488
[OK]   36/1488
[OK]   37/1488
[OK]   38/1488
[OK]   39/1488
[OK]   40/1488
[OK]   41/1488
[OK]   42/1488
[OK]   43/1488
[OK]   44/1488
[OK]   45/1488
[OK]   46/1488
[OK]   47/1488
[OK]   48/1488
[OK]   49/1488
[OK]   50/1488
💾 Đã lưu tạm 50 records
[OK]   51/1488
[OK]   52/1488
[OK]   53/1488
[OK]   54/1488
[OK]   55/1488
[OK]   56/1488
[OK]   57/1488
[OK]   58/1488
[OK]   59/1488
[OK]   60/1488
[OK]   61/1488
[OK]   62/1488
[OK]   63/1488
[OK]   64/1488
[OK]   65/1488
[OK]   66/

In [15]:
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print("✅ DONE – crawl full description")

✅ DONE – crawl full description
